# 1. How to init any model in one line

## Basic usage

In [1]:
from langchain.chat_models import init_chat_model

gemini = init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0)

print("Gemini: " + gemini.invoke("what's your name?").content)

Gemini: I do not have a name. I am a large language model, trained by Google.


##  Inferring model provider

## Creating a configurable model

In [3]:
configurable_model = init_chat_model(temperature=0)

configurable_model.invoke(
  "what's your name", config={"configurable": {"model": "gemini-2.5-flash", "model_provider": "google_genai"}}
)

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4087807a-4a3d-47e6-ba42-2af48ebfba90-0', usage_metadata={'input_tokens': 6, 'output_tokens': 150, 'total_tokens': 156, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 139}})

### Configurable model with default values

In [8]:
first_llm = init_chat_model(
  model="gemini-2.5-flash",
  model_provider="google_genai",
  temperature=0,
  configurable_fields=("model", "model_provider", "temperature", "max_tokens"),
  config_prefix="first"
)

first_llm.invoke("what's your name?")

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--45868077-2a0d-4ce5-b9d3-1695b9f724f9-0', usage_metadata={'input_tokens': 7, 'output_tokens': 53, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})

In [9]:
first_llm.invoke(
  "what's your name",
  config={
    "configurable": {
      "first_model": "gemini-2.5-flash-lite",
      "first_temperature": 0.8,
      "first_max_tokens:": 100
    }
  }
)

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--adccc4b4-f186-414f-8672-734d158d1e5f-0', usage_metadata={'input_tokens': 6, 'output_tokens': 18, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

### Using a configurable model declaratively

In [10]:
from pydantic import BaseModel, Field

class GetWeather(BaseModel):
   """Get the current weather in a given location"""
   location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

class GetPopulation(BaseModel):
  """Get the current population in a given location"""
  location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

llm = init_chat_model(temperature=0)
llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])

llm_with_tools.invoke(
  "what's bigger in 2024 LA or NYC", config={"configurable": {"model": "gemini-2.5-flash", "model_provider": "google_genai"}}
).tool_calls

[{'name': 'GetPopulation',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'd2ccb0a3-193b-45bf-9fea-40b26b41ad64',
  'type': 'tool_call'},
 {'name': 'GetPopulation',
  'args': {'location': 'New York, NY'},
  'id': '6298c11f-9450-4aa9-a175-98d022dad284',
  'type': 'tool_call'}]

# 2. Run models locally

## Quickstart

In [12]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gpt-oss:20b", validate_model_on_init=True) 
llm.invoke("The first man on the moon was ...").content

'Neil Armstrong.'

In [13]:
for chunk in llm.stream("The first man on the moon was ..."):
  print(chunk, end="|", flush=True)

content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|content='' additional_kwargs={} response_metadata={} id='run--ed3e007f-6f02-4259-8e41-f8ebe117d269'|

KeyboardInterrupt: 

In [ ]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.1:8b")
chat_model.invoke("Who was the first man on the moon?")

# 3. How to use chat models to call tools

## Defining tool schemas

### Python functions

In [14]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

### Langchain Tool

### Pydantic class

In [15]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

### TypedDict class

In [16]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [18]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
llm_with_tools = llm.bind_tools(tools)

query = "what is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 12.0, "a": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9a4da526-319a-4525-80c4-7b6e30a814a3-0', tool_calls=[{'name': 'multiply', 'args': {'b': 12.0, 'a': 3.0}, 'id': '605c764b-12e8-46a9-82ee-72766a81572f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 95, 'total_tokens': 199, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})

## Tool calls

In [19]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'b': 12.0, 'a': 3.0},
  'id': 'a18dded5-962b-4822-ba88-c3920c2f73fe',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'b': 49.0, 'a': 11.0},
  'id': '55991c6d-f071-4e34-b3eb-eae4bf1b1322',
  'type': 'tool_call'}]

## Parsing

In [ ]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12)]

# 3. How to return structured data from a model 

# 4. How to cache chat model responses

In [21]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [22]:
from langchain_core.globals import set_llm_cache

## In Memory Cache